In [1]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY")

print(api_key)

sk-bef7bd0c23094cecbbeff193c9e136ba


In [2]:
from glob import glob

text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [3]:
len(text_lines)

30

In [4]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1"
)

In [5]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from pymilvus import model as milvus_model
embedding_model = milvus_model.DefaultEmbeddingFunction()

/home/hopewang123456/conda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [10]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_mfd_collection"

In [11]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [12]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",
    consistency_level="Strong",
)

In [13]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 172842.20it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

In [14]:
question1 = "我们小区的垃圾桶被小孩弄坏，应该如何用法律教育他们？"

In [15]:
question2 = "父母留给我的遗产被银行抢占，说未能提供父母是我父母的法律条文，怎么办？"

In [17]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question1,question2]
    ),
    limit=3,
    search_params={"metric_type": "IP","params":{}},
    output_fields=["text"],
)

In [18]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u4e8c\u3001\u6743\u5229\u8d28\u6743\n\n**\u7b2c\u56db\u767e\u56db\u5341\u4e5d\u6761** \u53ef\u4ee5\u51fa\u8d28\u7684\u6743\u5229\u5305\u62ec\uff1a\n\uff08\u4e00\uff09\u6c47\u7968\u3001\u672c\u7968\u3001\u652f\u7968\uff1b\n\uff08\u4e8c\uff09\u503a\u5238\u3001\u5b58\u6b3e\u5355\uff1b\n\uff08\u4e09\uff09\u4ed3\u5355\u3001\u63d0\u5355\uff1b\n\uff08\u56db\uff09\u53ef\u4ee5\u8f6c\u8ba9\u7684\u57fa\u91d1\u4efd\u989d\u3001\u80a1\u6743\uff1b\n\uff08\u4e94\uff09\u53ef\u4ee5\u8f6c\u8ba9\u7684\u6ce8\u518c\u5546\u6807\u4e13\u7528\u6743\u3001\u4e13\u5229\u6743\u3001\u8457\u4f5c\u6743\u7b49\u77e5\u8bc6\u4ea7\u6743\u4e2d\u7684\u8d22\u4ea7\u6743\uff1b\n\uff08\u516d\uff09\u5e94\u6536\u8d26\u6b3e\uff1b\n\uff08\u4e03\uff09\u6cd5\u5f8b\u3001\u884c\u653f\u6cd5\u89c4\u89c4\u5b9a\u53ef\u4ee5\u51fa\u8d28\u7684\u5176\u4ed6\u8d22\u4ea7\u6743\u5229\u3002\n\n**\u7b2c\u56db\u767e\u4e94\u5341\u6761** \u4ee5\u6c47\u7968\u3001\u672c\u7968\u3001\u652f\u7968\u3001\u503a\u5238\u3001\u5b58\u6b3e\u5355\u3

In [19]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [20]:
context

'二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意

In [29]:
SYSTEM_PROMPT = """
Human: 你是一个优秀的 AI 法律助手。你能够从提供的上下文段落片段中找到问题的答案，并给予我合理的反馈信息。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段回答用 <questions> 标签括起来的不同问题。最后分开追加原始回答的中文建议在output标签中，分别在不同的question标签后用 <suggestion>和</suggestion> 标签标注建议和具体的<context>标签中标注的法律条文，全文均用中文作答。
<context>
{context}
</context>
<questions>
{question1},{question2}
</questions>
<output>
  <question>
   {question1}   
  </question>
  <suggestion>
  </suggestion>
  <question>
  {question2}  
  </question>
  <suggestion>
  </suggestion> 
</output>
"""

In [30]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段回答用 <questions> 标签括起来的不同问题。最后分开追加原始回答的中文建议在output标签中，分别在不同的question标签后用 <suggestion>和</suggestion> 标签标注建议和具体的<context>标签中标注的法律条文，全文均用中文作答。\n<context>\n二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质

In [31]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

<output>
  <question>
   我们小区的垃圾桶被小孩弄坏，应该如何用法律教育他们？   
  </question>
  <suggestion>
建议依据《民法典》侵权责任编相关规定处理：
1. 监护人责任：根据第一千一百八十八条，无民事行为能力人、限制民事行为能力人造成他人损害的，由监护人承担侵权责任。可要求其父母赔偿垃圾桶维修费用。
2. 财产保护：根据第一千一百六十五条，行为人因过错侵害他人民事权益造成损害的，应当承担侵权责任。可通过物业公司向监护人主张财产损害赔偿。
3. 教育方式：建议联合社区、学校开展普法教育，讲解《治安管理处罚法》第四十九条关于故意损毁公私财物的规定，培养未成年人法治意识。
  </suggestion>

  <question>
  父母留给我的遗产被银行抢占，说未能提供父母是我父母的法律条文，怎么办？  
  </question>
  <suggestion>
建议采取以下法律措施：
1. 举证责任：根据《民法典》继承编第一千一百三十九条，继承人应当提供继承权证明。需准备：
   - 亲属关系证明（户籍档案/出生医学证明）
   - 父母死亡证明
   - 遗嘱或法定继承文件

2. 权利救济：
   - 依据第一千一百四十二条，向法院提起继承权确认之诉
   - 根据第一千一百六十七条，可申请财产保全防止银行处置遗产

3. 特别提示：
   - 银行存款继承需依据《储蓄管理条例》办理继承公证
   - 若银行拒不配合，可向银保监会投诉（《商业银行法》第二十九条保护存款人合法权益）

注：建议委托律师调取户籍登记等法定证据，此类材料具有最高证明效力。
  </suggestion> 
</output>
